# Mean model
just submit average values 'lev_l' of each point through the time

In [1]:
import os
import numpy as np
import pandas as pd
import datetime
# import matplotlib.pyplot as plt
# %matplotlib inline


In [2]:
input_dir = '../input/'
working_dir = '../working/'
output_dir = '../output/'

## Read data

In [18]:
def read_track(line_name='a'):
    track_df = pd.read_csv(os.path.join(input_dir, 'track_{}.csv'.format(line_name.upper())))
    col_names_track = ['date', 'kilo', 'lev_l', 'lev_r', 'cur_l', 'cur_r', 'cant', 'width', 'speed']
    track_df.columns = col_names_track
    track_df['date'] = pd.to_datetime(track_df['date'])
    print('track_{line_name} shape: {shape}'.format(line_name=line_name.upper(), shape=track_df.shape))
    return track_df

In [16]:
def read_equ(line_name='a'):
    equ_df = pd.read_csv(os.path.join(input_dir, 'equipment_{}.csv'.format(line_name.upper())))
    col_names_equ = ['kilo', 'is_ballast', 'is_long', 'sleeper_type', 'is_bridge', 'is_crossing', 'gross_ton', 'radius', 'is_unreliable']
    equ_df.columns = col_names_equ
    print('equ_{line_name} shape: {shape}'.format(line_name=line_name.upper(), shape=equ_df.shape))
    return equ_df

In [46]:
abcd_list = ['a', 'b', 'c', 'd']
track = {}
equ = {}
for abcd in abcd_list:
    track[abcd] = read_track(abcd)
    equ[abcd] = read_equ(abcd)

track_A shape: (10185690, 9)
equ_A shape: (27906, 9)
track_B shape: (7815753, 9)
equ_B shape: (21531, 9)
track_C shape: (20324660, 9)
equ_C shape: (55684, 9)
track_D shape: (5601687, 9)
equ_D shape: (15691, 9)


In [43]:
sample_submit = pd.read_csv(os.path.join(input_dir, 'sample_submit.csv'), header=None)
index_master = pd.read_csv(os.path.join(input_dir, 'index_master.csv'))
index_master.columns = ['id', 'line_name', 'date', 'kilo']

## calculate mean as predicted values

In [97]:
track_pred = {}
track_pred_concat = pd.DataFrame()

In [98]:
for abcd in abcd_list:
    track_pred[abcd] = track[abcd].groupby('kilo')['lev_l'].mean()
    track_pred[abcd] = track_pred[abcd].to_frame()
    track_pred[abcd]['line_name'] = abcd.upper()
    track_pred[abcd] = track_pred[abcd].reset_index()
    track_pred[abcd]['kilo'] = track_pred[abcd]['kilo'].astype('int')
    track_pred_concat = pd.concat([track_pred_concat, track_pred[abcd]], axis=0)

In [100]:
submit = pd.merge(index_master, track_pred_concat, how='left', on=['kilo', 'line_name'])

In [102]:
submit[['id', 'lev_l']].to_csv(os.path.join(output_dir, 'mean_model_20181026.csv'), header=False, index=False)